# Soccer Project Revamp Scraping

### Read-In Important Players DF

* https://www.transfermarkt.us/schnellsuche/ergebnis/schnellsuche?query=berge
* https://www.bdfutbol.com/en/t/t2020-212035.html
* 2018-19 Tottenham had no impactful transfers

In [1]:
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests
import unidecode
import mysql.connector

In [2]:
# Reading in Players DF
important_players = pd.read_csv('revamp_important_players.csv', encoding = 'latin')
important_players.head()

,Season,Team,Starter #1,Starter #2,Starter #3,Starter #4,Starter #5,Starter #6,Starter #7,Starter #8,Starter #9,Starter #10,Starter #11,Substitute #1,Substitute #2,Substitute #3,Substitute #4,Substitute #5
0,2018-2019,Arsenal,Bernd Leno/GK/25,Sead Kolasinac/LB,Shkodran Mustafi/CB,Sokratis Papastathopoulos/CB/16,Nacho Monreal/RB,Granit Xhaka/DM,Lucas Torreira/DM/28.65,Matteo Guendouzi/AM/8,Pierre-Emerick Aubameyang/LW,Alexandre Lacazette/ST,Alex Iwobi/RW,Henrikh Mkhitaryan/RW,Mesut Ozil/AM,Hector Bellerin/LB,Laurent Koscielny/CB,Aaron Ramsey/DM
1,2018-2019,Bournemouth,Asmir Begovic/GK,Charlie Daniels/LB,Nathan Ake/CB,Steve Cook/CB,Adam Smith/RB,Jefferson Lerma/DM/30.5,Dan Gosling/DM,Ryan Fraser/RW,David Brooks/LW/11.3,Joshua King/ST,Callum Wilson/ST,Nathaniel Clyne/RB/loan,Andrew Surman/DM,Junior Stanislas/AM,Simon Francis/RB,Jordon Ibe/RW
2,2018-2019,Brighton & Hove Albion,Mathew Ryan/GK,Gaetan Bong/LB,Lewis Dunk/CB,Shane Duffy/CB,Martin Montoya/RB/7,Davy Propper/DM,Dale Stephens/DM,Pascal Gross/AM,Solly March/AM,Bernardo/LW/10,Glenn Murray/ST,Bruno/RB,Yves Bissouma/DM/16.8,Anthony Knockaert/RW,Jurgen Locadia/ST,Alireza Jahanbakhsh/RW/19
3,2018-2019,Burnley,Tom Heaton/GK,Charlie Taylor/LB,Ben Mee/CB,James Tarkowski/CB,Matthew Lowton/RB,Jack Cork/DM,Ashley Westwood/DM,Jeff Hendrick/AM,Johann Berg Gudmundsson/RW,Ashley Barnes/LW,Chris Wood/ST,Joe Hart/GK/3.9,Dwight McNeil/LW,Phil Bardsley/RB,Aaron Lennon/RW,Sam Vokes/ST
4,2018-2019,Cardiff City,Neil Etheridge/GK,Joe Bennett/LB,Bruno Ecuele Manga/CB,Sean Morrison/CB,Sol Bamba/CB,Aron Gunnarsson/DM,Harry Arter/AM/loan,Joe Ralls/DM,Victor Camarasa/AM/loan,Junior Hoilett/LW,Josh Murphy/RW/11.4,Callum Paterson/RB,Bobby Reid/AM/11.35,Lee Peltier/RB,Nathaniel Mendez-Laing/RW,Oumar Niasse/ST/loan


In [3]:
# Getting player

def get_player_year_combo(row):
    return_list = []
    season = row['Season']
    team = row['Team']
    for i in range(1, 12):
        player = row[f"Starter #{i}"].split('/')[0]
        player_pos = row[f"Starter #{i}"].split('/')[1]
        try:
            player_cost = row[f"Starter #{i}"].split('/')[2]
        except:
            player_cost = pd.NA
        return_list.append([player, season, team, player_pos, player_cost])
    
    for i in range(1, 6):
        player = row[f"Substitute #{i}"].split('/')[0]
        player_pos = row[f"Substitute #{i}"].split('/')[1]
        try:
            player_cost = row[f"Substitute #{i}"].split('/')[2]
        except:
            player_cost = pd.NA
        return_list.append([player, season, team, player_pos, player_cost])
        
    return return_list

### Maybe change csv file so its name/position/cost

In [4]:
player_series = important_players.apply(get_player_year_combo, axis = 1)
extended_list = player_series.tolist()
player_list = []
for team_season in extended_list:
    for player in team_season:
        player_list.append(player)
#player_list

In [5]:
df = pd.DataFrame(player_list, columns = ['name', 'season', 'team', 'position', 'transfer info'])
df

,name,season,team,position,transfer info
0,Bernd Leno,2018-2019,Arsenal,GK,25
1,Sead Kolasinac,2018-2019,Arsenal,LB,<NA>
2,Shkodran Mustafi,2018-2019,Arsenal,CB,<NA>
3,Sokratis Papastathopoulos,2018-2019,Arsenal,CB,16
4,Nacho Monreal,2018-2019,Arsenal,RB,<NA>
...,...,...,...,...,...
1595,Mario Lemina,2022-2023,Wolverhampton Wanderers,AM,<NA>
1596,Hwang Hee-chan,2022-2023,Wolverhampton Wanderers,LW,<NA>
1597,Hugo Bueno,2022-2023,Wolverhampton Wanderers,LB,<NA>
1598,Jonny Castro,2022-2023,Wolverhampton Wanderers,RB,<NA>


## Pre-Scraping Functions

In [6]:
cnx = mysql.connector.connect(user = 'wsa',
                             host = '34.68.250.121',
                             database = 'Soccer-2023',
                             password = 'LeBron>MJ!')
cursor = cnx.cursor(buffered = True)

In [7]:
# check if the gks i'm interested in were all found by the scraper
def found_all(found_list, wanted_list):
    missing = []
    for player in wanted_list:
        if player in found_list:
            continue
        else:
            missing.append(player)
    return missing

In [8]:
# Winter transfers whose stats we need to block because they get in the way of finding data I want
winters = [['Mathew Ryan', '2020-2021', 'Arsenal'], ['Nathaniel Clyne', '2018-2019', 'Liverpool'], ['Oumar Niasse', '2018-2019', 'Everton'],
           ['Ryan Bennett', '2019-2020', 'Leicester City'], ['Cedric Soares', '2019-2020', 'Arsenal'], ['Ross Barkley', '2020-2021', 'Chelsea'],
           ['Ruben Loftus-Cheek', '2020-2021', 'Chelsea'], ['Ainsley Maitland-Niles', '2020-2021', 'Arsenal'], ['Theo Walcott', '2020-2021', 'Everton'],
           ['Dan Burn', '2021-2022', 'Brighton'], ['Lucas Digne', '2021-2022', 'Everton'], ['Daniel James', '2021-2022', 'Manchester Utd'],
           ['Matt Targett', '2021-2022', 'Newcastle Utd'], ['Chris Wood', '2021-2022', 'Burnley'], ['Jan Bednarek', '2022-2023', 'Aston Villa'],
           ['Craig Dawson', '2022-2023', 'West Ham'], ['Morgan Gibbs-White', '2022-2023', 'Wolves'], ['Anthony Gordon', '2022-2023', 'Everton'],
           ['Danny Ings', '2022-2023', 'West Ham'], ['Jorginho', '2022-2023', 'Chelsea'],['Leandro Trossard', '2022-2023', 'Arsenal']]

## Scrape GK Data

Basic Steps
1. Access FBref page for basic and advanced stats
2. If the html row (player) is in our list of name/seasons... get their stats

In [10]:
# Get list of GKs that I am trying to find
gks = df.groupby('position').get_group('GK')
gk_names = gks['name'].tolist()
gk_seasons = gks['season'].tolist()
wanted_gk = []
for i in range(len(gk_names)):
    wanted_gk.append([gk_names[i], gk_seasons[i]])
#wanted_gk

In [11]:
# SCRAPING BASIC GK STATS
prem_years = ['2018-2019', '2019-2020', '2020-2021', '2021-2022', '2022-2023']
found_gk_basics_list = []
gk_values = []
for year in prem_years:
    season = year
    basic_url = requests.get(f"https://fbref.com/en/comps/9/{year}/keepers/{year}-Premier-League-Stats")
    basic_soup = BeautifulSoup(basic_url.text, 'html.parser')
    block = [i for i in basic_soup.find_all(string=lambda text: isinstance(text, bs4.Comment)) if 'id="div_stats_keeper"' in i][0]
    basic_soup = BeautifulSoup(str(block), 'html.parser')
    basic_table = basic_soup.find('table').find('tbody')
    basic_rows = basic_table.find_all('tr')
    for row in basic_rows:
        columns = row.find_all('td')
        if len(columns) > 0:
            name = unidecode.unidecode(columns[0].find('a').text)
            if [name, season] in wanted_gk:
                fbref_club = columns[3].find('a').text
                if [name, season, fbref_club] in winters:
                    continue
                found_gk_basics_list.append([name, season])
                ga = int(columns[10].text)
                ga90 = float(columns[11].text)
                sota = int(columns[12].text)
                saves = int(columns[13].text)
                savepct = float(columns[14].text)
                cs = int(columns[18].text)
                csper = float(columns[19].text)
                if columns[24].text == '':
                    pksaveper = 0.0
                else:
                    pksaveper = float(columns[24].text)
                club = df[(df['name'] == name) & (df['season'] == season)]['team'].tolist()[0]
                position = df[(df['name'] == name) & (df['season'] == season)]['position'].tolist()[0]
                transfer_info = df[(df['name'] == name) & (df['season'] == season)]['transfer info'].tolist()[0]
                if pd.isnull(transfer_info):
                    transfer_info = 'No transfer'
                values = [name, season, club, position, transfer_info, ga, ga90, sota, saves, savepct, cs, csper, pksaveper]
                gk_values.append(values)


In [12]:
# SCRAPING ADVANCED GK STATS
prem_years = ['2018-2019', '2019-2020', '2020-2021', '2021-2022', '2022-2023']
found_gk_adv = []
for year in prem_years:
    season = year
    adv_url = requests.get(f"https://fbref.com/en/comps/9/{year}/keepersadv/{year}-Premier-League-Stats")
    adv_soup = BeautifulSoup(adv_url.text, 'html.parser')
    block = [i for i in adv_soup.find_all(string=lambda text: isinstance(text, bs4.Comment)) if 'id="div_stats_keeper_adv"' in i][0]
    adv_soup = BeautifulSoup(str(block), 'html.parser')
    adv_table = adv_soup.find('table').find('tbody')
    adv_rows = adv_table.find_all('tr')
    for row in adv_rows:
        columns = row.find_all('td')
        if len(columns) > 0:
            name = unidecode.unidecode(columns[0].find('a').text)
            if [name, season] in wanted_gk:
                found_gk_adv.append([name, season])
                psxg = float(columns[12].text)
                psxg90 = float(columns[13].text)
                launch_cmp_per = float(columns[18].text)
                cross_stop_per = float(columns[28].text)
                for gk in gk_values:
                    if gk[0] == name and gk[1] == season:
                        fbref_club = columns[3].find('a').text
                        if [name, season, fbref_club] in winters:
                            continue
                        gk.extend([psxg, psxg90, launch_cmp_per, cross_stop_per])
                        statement = "INSERT INTO revamp_gk (name, season, club, position, transfer_info, GA, GA90, SoTA, saves, save_pct, clean_sheets, clean_sheets_pct, pk_save_pct, PSxG, PSxG90, launch_cmp_pct, cross_stop_pct) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                        #cursor.execute(statement, gk)
                        #cnx.commit()
               

In [13]:
# Showing all gk stats that I found
# for gk in gk_values:
#     print(gk)
print(len(gk_values))

114


In [14]:
# Check to see if I am missing the scraping data for each player that I want
print(found_all(found_gk_basics_list, wanted_gk))
print(found_all(found_gk_adv, wanted_gk))

[]
[]


## Scrape Outfield Player Data

Basic Steps
1. Access FBref page for basic and advanced stats
2. If the html row (player) is in our list of name/seasons... get their stats

In [15]:
# Get list of players that I am trying to find
players = df[df['position'] != 'GK']
player_names = players['name'].tolist()
player_seasons = players['season'].tolist()
wanted_players = []
for i in range(len(player_names)):
    wanted_players.append([player_names[i], player_seasons[i]])
#wanted_players

In [16]:
# SCRAPING PLAYER STANDARD STATS
prem_years = ['2018-2019', '2019-2020', '2020-2021', '2021-2022', '2022-2023']
found_player_standard = []
player_values = []
for year in prem_years:
    season = year
    url = requests.get(f"https://fbref.com/en/comps/9/{year}/stats/{year}-Premier-League-Stats")
    soup_blocked = BeautifulSoup(url.text, 'html.parser')
    block = [i for i in soup_blocked.find_all(string=lambda text: isinstance(text, bs4.Comment)) if 'id="div_stats_standard"' in i][0]
    soup = BeautifulSoup(str(block), 'html.parser')
    table = soup.find('table').find('tbody')
    rows = table.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        if len(columns) > 0:
            name = unidecode.unidecode(columns[0].find('a').text)
            if [name, season] in wanted_players:
                fbref_club = columns[3].find('a').text
                if [name, season, fbref_club] in winters:
                    continue
                found_player_standard.append([name, season])
                goals = int(columns[10].text)
                assists = int(columns[11].text)
                crdY = int(columns[16].text)
                crdR = int(columns[17].text)
                xG = float(columns[18].text)
                npxG = float(columns[19].text)
                xA = float(columns[20].text)
                progC = int(columns[22].text)
                progP = int(columns[23].text)
                progR = int(columns[24].text)
                club = df[(df['name'] == name) & (df['season'] == season)]['team'].tolist()[0]
                position = df[(df['name'] == name) & (df['season'] == season)]['position'].tolist()[0]
                transfer_info = df[(df['name'] == name) & (df['season'] == season)]['transfer info'].tolist()[0]
                if pd.isnull(transfer_info):
                    transfer_info = 'No transfer'
                standard_vals = [name, season, club, position, transfer_info, goals, assists, crdY, crdR, xG, npxG, xA, progC, progP, progR]
                player_values.append(standard_vals)
                #print(values)
        

In [17]:
# SCRAPING PLAYER SHOOTING STATS
prem_years = ['2018-2019', '2019-2020', '2020-2021', '2021-2022', '2022-2023']
found_player_shooting = []
for year in prem_years:
    season = year
    url = requests.get(f"https://fbref.com/en/comps/9/{year}/shooting/{year}-Premier-League-Stats")
    soup_blocked = BeautifulSoup(url.text, 'html.parser')
    block = [i for i in soup_blocked.find_all(string=lambda text: isinstance(text, bs4.Comment)) if 'id="div_stats_shooting"' in i][0]
    soup = BeautifulSoup(str(block), 'html.parser')
    table = soup.find('table').find('tbody')
    rows = table.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        if len(columns) > 0:
            name = unidecode.unidecode(columns[0].find('a').text)
            if [name, season] in wanted_players:
                found_player_shooting.append([name, season])
                shots = int(columns[8].text)
                sot = int(columns[9].text)
                if columns[13].text == '':
                    g_per_shot = 0.00
                else:
                    g_per_shot = float(columns[13].text)
                shooting_vals = [shots, sot, g_per_shot]
                for player in player_values:
                    if player[0] == name and player[1] == season:
                        fbref_club = columns[3].find('a').text
                        if [name, season, fbref_club] in winters:
                            continue
                        player.extend(shooting_vals)

In [18]:
# SCRAPING PLAYER PASSING STATS
prem_years = ['2018-2019', '2019-2020', '2020-2021', '2021-2022', '2022-2023']
found_player_passing = []
for year in prem_years:
    season = year
    url = requests.get(f"https://fbref.com/en/comps/9/{year}/passing/{year}-Premier-League-Stats")
    soup_blocked = BeautifulSoup(url.text, 'html.parser')
    block = [i for i in soup_blocked.find_all(string=lambda text: isinstance(text, bs4.Comment)) if 'id="div_stats_passing"' in i][0]
    soup = BeautifulSoup(str(block), 'html.parser')
    table = soup.find('table').find('tbody')
    rows = table.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        if len(columns) > 0:
            name = unidecode.unidecode(columns[0].find('a').text)
            if [name, season] in wanted_players:
                found_player_passing.append([name, season])
                comp_passes = int(columns[7].text)
                pass_cmp_pct = float(columns[9].text)
                passing_vals = [comp_passes, pass_cmp_pct]
                for player in player_values:
                    if player[0] == name and player[1] == season:
                        fbref_club = columns[3].find('a').text
                        if [name, season, fbref_club] in winters:
                            continue
                        player.extend(passing_vals)

In [19]:
# SCRAPING PLAYER PASSING TYPES STATS
prem_years = ['2018-2019', '2019-2020', '2020-2021', '2021-2022', '2022-2023']
found_player_passtypes = []
for year in prem_years:
    season = year
    url = requests.get(f"https://fbref.com/en/comps/9/{year}/passing_types/{year}-Premier-League-Stats")
    soup_blocked = BeautifulSoup(url.text, 'html.parser')
    block = [i for i in soup_blocked.find_all(string=lambda text: isinstance(text, bs4.Comment)) if 'id="div_stats_passing_types"' in i][0]
    soup = BeautifulSoup(str(block), 'html.parser')
    table = soup.find('table').find('tbody')
    rows = table.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        if len(columns) > 0:
            name = unidecode.unidecode(columns[0].find('a').text)
            if [name, season] in wanted_players:
                found_player_passtypes.append([name, season])
                through_balls = int(columns[11].text)
                switches = int(columns[12].text)
                crosses = int(columns[13].text)
                passtypes_vals = [through_balls, switches, crosses]
                for player in player_values:
                    if player[0] == name and player[1] == season:
                        fbref_club = columns[3].find('a').text
                        if [name, season, fbref_club] in winters:
                            continue
                        player.extend(passtypes_vals)

In [20]:
# SCRAPING PLAYER GOAL AND SHOT CREATION STATS
prem_years = ['2018-2019', '2019-2020', '2020-2021', '2021-2022', '2022-2023']
found_player_gca = []
for year in prem_years:
    season = year
    url = requests.get(f"https://fbref.com/en/comps/9/{year}/gca/{year}-Premier-League-Stats")
    soup_blocked = BeautifulSoup(url.text, 'html.parser')
    block = [i for i in soup_blocked.find_all(string=lambda text: isinstance(text, bs4.Comment)) if 'id="div_stats_gca"' in i][0]
    soup = BeautifulSoup(str(block), 'html.parser')
    table = soup.find('table').find('tbody')
    rows = table.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        if len(columns) > 0:
            name = unidecode.unidecode(columns[0].find('a').text)
            if [name, season] in wanted_players:
                found_player_gca.append([name, season])
                sca = int(columns[7].text)
                gca = int(columns[15].text)
                gca_vals = [sca, gca]
                for player in player_values:
                    if player[0] == name and player[1] == season:
                        fbref_club = columns[3].find('a').text
                        if [name, season, fbref_club] in winters:
                            continue
                        player.extend(gca_vals)

In [21]:
# SCRAPING PLAYER DEFENSIVE ACTIONS STATS
prem_years = ['2018-2019', '2019-2020', '2020-2021', '2021-2022', '2022-2023']
found_player_def = []
for year in prem_years:
    season = year
    url = requests.get(f"https://fbref.com/en/comps/9/{year}/defense/{year}-Premier-League-Stats")
    soup_blocked = BeautifulSoup(url.text, 'html.parser')
    block = [i for i in soup_blocked.find_all(string=lambda text: isinstance(text, bs4.Comment)) if 'id="div_stats_defense"' in i][0]
    soup = BeautifulSoup(str(block), 'html.parser')
    table = soup.find('table').find('tbody')
    rows = table.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        if len(columns) > 0:
            name = unidecode.unidecode(columns[0].find('a').text)
            if [name, season] in wanted_players:
                found_player_def.append([name, season])
                tackles = int(columns[7].text)
                if columns[14].text == '':
                    tackle_win_pct = 0.0
                else:
                    tackle_win_pct = float(columns[14].text)
                blocks = int(columns[16].text)
                ints = int(columns[19].text)
                clearances = int(columns[21].text)
                errors = int(columns[22].text)
                def_vals = [tackles, tackle_win_pct, blocks, ints, clearances, errors]
                for player in player_values:
                    if player[0] == name and player[1] == season:
                        fbref_club = columns[3].find('a').text
                        if [name, season, fbref_club] in winters:
                            continue
                        player.extend(def_vals)

In [22]:
# SCRAPING PLAYER POSSESSION STATS
prem_years = ['2018-2019', '2019-2020', '2020-2021', '2021-2022', '2022-2023']
found_player_poss = []
for year in prem_years:
    season = year
    url = requests.get(f"https://fbref.com/en/comps/9/{year}/possession/{year}-Premier-League-Stats")
    soup_blocked = BeautifulSoup(url.text, 'html.parser')
    block = [i for i in soup_blocked.find_all(string=lambda text: isinstance(text, bs4.Comment)) if 'id="div_stats_possession"' in i][0]
    soup = BeautifulSoup(str(block), 'html.parser')
    table = soup.find('table').find('tbody')
    rows = table.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        if len(columns) > 0:
            name = unidecode.unidecode(columns[0].find('a').text)
            if [name, season] in wanted_players:
                found_player_poss.append([name, season])
                take_ons = int(columns[15].text)
                if columns[16].text == '':
                    take_on_pct = 0.0
                else:
                    take_on_pct = float(columns[16].text)
                final_third_carries = int(columns[23].text)
                poss_vals = [take_ons, take_on_pct, final_third_carries]
                for player in player_values:
                    if player[0] == name and player[1] == season:
                        fbref_club = columns[3].find('a').text
                        if [name, season, fbref_club] in winters:
                            continue
                        player.extend(poss_vals)

In [23]:
# SCRAPING PLAYER MISC STATS
prem_years = ['2018-2019', '2019-2020', '2020-2021', '2021-2022', '2022-2023']
found_player_misc = []
for year in prem_years:
    season = year
    url = requests.get(f"https://fbref.com/en/comps/9/{year}/misc/{year}-Premier-League-Stats")
    soup_blocked = BeautifulSoup(url.text, 'html.parser')
    block = [i for i in soup_blocked.find_all(string=lambda text: isinstance(text, bs4.Comment)) if 'id="div_stats_misc"' in i][0]
    soup = BeautifulSoup(str(block), 'html.parser')
    table = soup.find('table').find('tbody')
    rows = table.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        if len(columns) > 0:
            name = unidecode.unidecode(columns[0].find('a').text)
            if [name, season] in wanted_players:
                found_player_misc.append([name, season])
                fls_c = int(columns[10].text)
                fls_d = int(columns[11].text)
                ball_rec = int(columns[19].text)
                aerials_won = int(columns[20].text)
                if columns[22].text == '':
                    aerials_pct = 0.0
                else:
                    aerials_pct = float(columns[22].text)
                misc_vals = [fls_c, fls_d, ball_rec, aerials_won, aerials_pct,]
                for player in player_values:
                    if player[0] == name and player[1] == season:
                        fbref_club = columns[3].find('a').text
                        if [name, season, fbref_club] in winters:
                            continue
                        player.extend(misc_vals)

In [30]:
for player in player_values:
    print(len(player))
# Missing 4 guys from the scraping

39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
3

In [27]:
print(len(wanted_gk))
print(len(wanted_players))
print(len(gk_values))
print(len(player_values))

114
1486
114
1486


In [24]:
# Check to see if I am missing the scraping data for each player that I want
print(found_all(found_player_standard, wanted_players))
print(found_all(found_player_shooting, wanted_players))
print(found_all(found_player_passing, wanted_players))
print(found_all(found_player_passtypes, wanted_players))
print(found_all(found_player_gca, wanted_players))
print(found_all(found_player_def, wanted_players))
print(found_all(found_player_poss, wanted_players))
print(found_all(found_player_misc, wanted_players))

[]
[]
[]
[]
[]
[]
[]
[]


In [32]:
for player in player_values:
    statement = "INSERT INTO revamp_players (name, season, club, position, transfer_info, goals, assists, crdY, crdR, xG, npxG, xA, progC, progP, progR, shots, SoT, g_per_shot, comp_passes, pass_cmp_pct, through_balls, switches, crosses, sca, gca, tackles, tackle_win_pct, blocks, ints, clearances, errors, take_ons, take_on_pct, final_third_carries, fls_c, fls_d, ball_rec, aerials_won, aerials_pct) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(statement, player)
    cnx.commit()